# 🔌 Plugins 101

## 🏁 Let's kick this off with the right packages

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.1.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.1.0-alpha"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"

Installed Packages Microsoft.Extensions.Logging.Console, 8.0.0 Microsoft.SemanticKernel, 1.1.0 Microsoft.SemanticKernel.Plugins.Core, 1.1.0-alpha

## 🔥 Fire up the kernel

In [ ]:
#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Extensions.Logging;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI) {
    kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey)
        .Build();
} else {
    kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion("gpt-4-1106-preview", apiKey, orgId)
        .Build();
}

## ⌚️ Let's use a native time function from the .NET world

In [ ]:
using System.ComponentModel;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;

public class TimeInformationPlugin
{
    [KernelFunction]
    [Description("Retrieves the current time in UTC.")]
    public string GetCurrentUtcTime() => DateTime.UtcNow.ToString("R");
}

kernel.ImportPluginFromType<TimeInformationPlugin>();

Semantic code lives in the "mind" of the LLM. We can ground the AI in knowledge of the world outside its neural networks to know about today's date just by giving it a plugin that can access native code capabilities.

## 🪴 This simple bit of "grounding" reduces so-called 'hallucination' with LLMs

In this first example, we invoke the kernel with a prompt that asks the AI for information it cannot provide. It has no idea what today might be for you, so it's likely to hallucinate.

In [ ]:
Console.WriteLine(await kernel.InvokePromptAsync("How many days until Christmas?"));

We can do this with streaming, too, and keep in mind that sometimes the model is able to know what today is ... somehow.

In [ ]:
// In case you want streaming, you could do it this way too
OpenAIPromptExecutionSettings settings = new() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };
await foreach (var update in kernel.InvokePromptStreamingAsync("How many days until Christmas?", new(settings)))
{
    Console.Write(update);
}

In this second example, the plugin we've added `TimeInformationPlugin` is used in the templated prompt. And by doing so, you can see the LLM thinking out loud as based upon the right information to start from. By giving it a native plugin that talks to your computer, it can ascertain the time of day like a normal function that would be called on your computer.

In [ ]:
Console.WriteLine(await kernel.InvokePromptAsync("The current time is {{TimeInformationPlugin.GetCurrentUtcTime}}. How many days until Christmas?"));

In this last example, we'll invoke the kernel with a prompt and allow the AI to automatically invoke functions. Note that we're hoping it will find `TimeInformationPlugin` -- but keep in mind that sometimes the LLM will 'hallucinate' the existence of a plugin that isn't out there. Weird, right?

In [ ]:
OpenAIPromptExecutionSettings settings = new() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };
Console.WriteLine(await kernel.InvokePromptAsync("How many days until Christmas? Explain your thinking.", new(settings)));

## 🔌 Let's add another plugin, but this time show how to make a hybrid plugin

In [ ]:
using Microsoft.SemanticKernel.Connectors.OpenAI;

public class ExampleComboPlugin
{
    // THIS IS A NATIVE FUNCTION WITH NO ARGUMENTS
    [KernelFunction]
    [Description("Retrieves a quote from John Maeda")]
    public string GetJMQuote() => "The arts are the science of enjoying life.";

    // THIS IS A NATIVE FUNCTION WITH ARGUMENTS
    [KernelFunction, Description("When bringing up math, John Maeda would say something like this")]
    public string JMSaysSomething(
        [Description("The topic that you want him to say something about")] string topic) =>
        $"John's not that clever but regarding '{topic}' he would definitely say how math and art go together.";

    // THIS IS A SEMANTIC FUNCTION WITH ARGUMENTS
    [KernelFunction, Description("Takes a text and generates a John Maeda idea")]
    public async Task<string> JMImaginesSomething(Kernel kernel, [Description("The topic that you want him to say something about")] string topic)
    {
        KernelArguments args = new(
            new OpenAIPromptExecutionSettings { MaxTokens = 500, Temperature = 0.5 }) { { "topic", topic } };
        var result = await kernel.InvokePromptAsync(
            @"In less than 140-characters, consider how John Maeda would think of how design and technology impact {{$topic}}. Output:", args); 
        return result.ToString();
    }
}

var comboPlugin = kernel.ImportPluginFromType<ExampleComboPlugin>();

With SK you can directly invoke each of the functions, let's show that in action by starting with the two native functions. First we'll get the quote from John (spoiler alert: it's a native function that just returns one string and doesn't get creative).

In [ ]:
Console.WriteLine(await kernel.InvokeAsync(comboPlugin["GetJMQuote"]));

Next we'll use the native function that takes an argument. Again, don't expect anything magical to happen because it's just good old fashioned native code.

In [ ]:
Console.WriteLine(await kernel.InvokeAsync(comboPlugin["JMSaysSomething"], new() { ["topic"] = "math" }));
Console.WriteLine(await kernel.InvokeAsync(comboPlugin["JMSaysSomething"], new() { ["topic"] = DateTime.Now}));

And for our last trick in this section, we're going to call the semantic function.

In [ ]:
Console.WriteLine(await kernel.InvokeAsync(comboPlugin["JMImaginesSomething"], new() { ["topic"] = "flamingos" }));

### ☎️ Prompt calling another prompt?

We can fluidly incorporate the results of one function into the input of another function within our prompt templating system. Here we use the semantic function to get creative, but then pass the input to the native function.

In [ ]:
Console.WriteLine(await kernel.InvokePromptAsync("Rewrite the following in less than 5 words: {{ExampleComboPlugin.JMImaginesSomething 'igloos'}}"));

### ☎️ Prompt calling a prompt and native code?

Keep in mind that the prompt templating language is quite flexible.

In [ ]:
Console.WriteLine(await kernel.InvokePromptAsync("Rewrite the following in less than 15 words: {{ExampleComboPlugin.JMImaginesSomething 'venture capital funding'}} {{ExampleComboPlugin.GetJMQuote}}"));

## 📋 Keep in mind that the kernel keeps track of all the plugins that you register with it

This code lets you browse the available plugins and related functions to your kernel instance.

In [ ]:
static void PrintAllPluginFunctions(Kernel kernel)
{
    var functions = kernel.Plugins.GetFunctionsMetadata();

    Console.WriteLine("****** Registered 🔌 Plugins and 📦 Functions ******");

    foreach (KernelFunctionMetadata func in functions)
    {
        PrintPluginFunction(func);
    }
}

static void PrintPluginFunction(KernelFunctionMetadata func)
{
    Console.WriteLine($"🔌 {func.PluginName}");
    Console.WriteLine($"   📦 /{func.Name}: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("      📥 Params:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"       • {p.Name}: {p.Description} (default: '{p.DefaultValue}')");
        }
    }
}

PrintAllPluginFunctions(kernel);

# 🔌 More plugins means the more it can go out and do for you

## 🔌 This is how you would include a native plugin in a .cs file

Note that you would say `using EmailPlugin` and inside the EmailPlugin.cs file you might want to set up a namespace like `namespace Plugins` but that's not what we do inside a Jupyter notebook as shown below.

In [ ]:
#!import Plugins/EmailPlugin.cs

var emailPlugin = kernel.ImportPluginFromType<EmailPlugin>();

PrintAllPluginFunctions(kernel);

## 🔌 This is an extremely compact, _quick_ way to make an all-native plugin

Note that the input variable name `cityName` becomes the descriptive text used to define the kind of input variable.

In [ ]:
kernel.ImportPluginFromFunctions("NanoPlugin", new[]
{
    kernel.CreateFunctionFromMethod(() => DateTime.UtcNow.ToString("R"), "GetCurrentUtcTime", "Retrieves the current time in UTC."),
    kernel.CreateFunctionFromMethod((string cityName) =>
        cityName switch
        {
            "Boston" => "61 and rainy",
            "London" => "55 and cloudy",
            "Miami" => "80 and sunny",
            "Paris" => "60 and rainy",
            "Tokyo" => "50 and sunny",
            "Sydney" => "75 and sunny",
            "Tel Aviv" => "80 and sunny",
            _ => "31 and snowing",
        }, "Get_Weather_For_City", "Gets the current weather for the specified city"),
});

PrintAllPluginFunctions(kernel);

## 📁 We can also point it to your Plugin folder

The planner needs to know what plugins are available to it. Here we'll import the `PracticePlugin` that has a semantic function inside it using the original SK prompt format.

In [ ]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "Plugins");

kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "PracticePlugin"));
PrintAllPluginFunctions(kernel);

## 👈 Let's take a look at the emerging YAML format for prompts

## 🏁 Let's grab the relevant packages

In [ ]:
#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Yaml, 1.0.1"

## 🥱 We can load the original prompt template format in YAML

In [ ]:
using System;
using System.Threading.Tasks;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.PromptTemplates.Handlebars;
using System.IO;

// Load prompt from resource
var generateStoryYaml = File.ReadAllText("Resources/GenerateStory.yaml");
var function = kernel.CreateFunctionFromPromptYaml(generateStoryYaml);

// Invoke the prompt function and display the result
Console.WriteLine(await kernel.InvokeAsync(function, arguments: new()
    {
        { "topic", "Dog" },
        { "length", "3" },
    }));


## 🥱🚲 And we can load it with the Handlebars option as well

In [ ]:
// Load prompt from resource
var generateStoryHandlebarsYaml = File.ReadAllText("Resources/GenerateStoryHandlebars.yaml");
function = kernel.CreateFunctionFromPromptYaml(generateStoryHandlebarsYaml, new HandlebarsPromptTemplateFactory());

// Invoke the prompt function and display the result
Console.WriteLine(await kernel.InvokeAsync(function, arguments: new()
    {
        { "topic", "Cat" },
        { "length", "3" },
    }));

## 🔌🤨 A plugin is essentially an organizing 🗂️ folder of functions


Keep in mind that they above functions are available to the kernel, but they're NOT registered as plugins. You can verify that by noticing the function isn't being registered as an available plugin for the kernel. 

In [ ]:
PrintAllPluginFunctions(kernel);

In Semantic Kernel, plugins can hold multiple functions — that's a unique benefit that only pays dividends later when you want to organize your functions into a nicely encapsulated folder, of sorts. Errr ... plugin. You get my gist, right?

## 🔌📦 You can turn a single function 📦 into a plugin — that's totally fine, too

Note that the input variable names become the descriptive text used to define what kinds of input are being fed into the function.

In [ ]:
var my1stFunctionAsPlugin = kernel.ImportPluginFromFunctions("My1stFunctionAsInstantPlugin", new[]
{
    kernel.CreateFunctionFromMethod((string itemName) => { return ($"Yes that item is {itemName} million dollars.");}, 
        "CalculateCost", 
        "Computes the cost of any named item."),
});

PrintAllPluginFunctions(kernel);

We can invoke the function directly because we're holding onto it in `my1stFunctionAsPlugin`:

In [ ]:
Console.WriteLine(await kernel.InvokeAsync(my1stFunctionAsPlugin["CalculateCost"], new() { ["itemName"] = "potato chips" }));

This is an example of doing that for a semantic function, or "templated prompt":

In [ ]:
var my2ndFunctionAsPlugin = kernel.ImportPluginFromFunctions("My2ndFunctionAsInstantPlugin", new[]
{
    kernel.CreateFunctionFromMethod(async (string topicDescription) => { 
         KernelArguments args = new(
            new OpenAIPromptExecutionSettings { MaxTokens = 500, Temperature = 0.5 }) { { "topic", topicDescription } };
        var result = await kernel.InvokePromptAsync(
            @"In less than 140-characters, consider how John Maeda would think of how design and technology impact {{$topic}}. Output:", args); 
        return result.ToString();},
        "MaedaTopicSentence", 
        "Create a topic sentence that John Maeda would likely use."),
});

PrintAllPluginFunctions(kernel);

We can invoke the function directly because we're holding onto it in `my2ndFunctionAsPlugin`:

In [ ]:
Console.WriteLine(await kernel.InvokeAsync(my2ndFunctionAsPlugin["MaedaTopicSentence"], new() { ["topicDescription"] = "potato chips" }));

Streaming tends to be the best way to invoke prompts, so:

In [ ]:
KernelArguments args = new(
    new OpenAIPromptExecutionSettings { MaxTokens = 500, Temperature = 0.5, ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions }) 
    { { "topicDescription", "potato chips" } };
await foreach (var update in kernel.InvokeStreamingAsync(my2ndFunctionAsPlugin["MaedaTopicSentence"], args))
{
    Console.Write(update);
}